In [3]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("../utils.jl")


print_percentiles (generic function with 2 methods)

In [35]:
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net_PLN, :MwigTR, :WIG20TR]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes_PLN"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_pln = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> from(x, Date(2003,01,01)) |>
            x -> percentchange(x, :log)

254×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-02-28 to 2024-03-31
┌────────────┬────────────┬──────────────┬────────────┬──────────────┬──────────
│            │ USA_Net    │ World_ex_USA │ EM_Net     │ ACWI_Net_PLN │ MwigTR  ⋯
├────────────┼────────────┼──────────────┼────────────┼──────────────┼──────────
│ 2003-02-28 │ -0.0159343 │   -0.0199094 │ -0.0284988 │   0.00510351 │ -0.0010 ⋯
│ 2003-03-31 │ 0.00886423 │   -0.0194601 │ -0.0289739 │    0.0430687 │  -0.002 ⋯
│ 2003-04-30 │  0.0798764 │    0.0916379 │  0.0851925 │    0.0118412 │   0.034 ⋯
│ 2003-05-31 │  0.0517369 │    0.0601744 │  0.0690563 │    0.0300993 │   0.055 ⋯
│ 2003-06-30 │   0.011892 │    0.0240024 │  0.0551391 │    0.0670022 │   0.016 ⋯
│ 2003-07-31 │  0.0179915 │    0.0226828 │  0.0603263 │    0.0132941 │    0.11 ⋯
│ 2003-08-31 │  0.0181089 │    0.0254588 │  0.0647735 │    0.0495565 │    0.16 ⋯
│ 2003-09-30 │ -0.0120244 │    0.0295601 │  0.0072564 │  0.000195407 │  -0.087 ⋯
│     ⋮      │     ⋮      │      

In [36]:
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │
├────────────┼────────────┼──────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │
│ 2023-10-31 │ -0.0235616 │   -0.0431263 │   -0.039629 │
│ 2023-11-30 │  0.0895259 │    0.0898198 │   0.0769883 │
│ 2023-12-31 │  0.0456572 │    0.0532219 │   0.0383487 │
│ 2024-01-31

In [37]:
returns_summarystats(returns_pln, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1495,1.4788,0.1,0.4071,-2.257,-0.7994,-0.1874,0.0375,1.9085,0.6692,0.1546
World_ex_USA,0.1681,1.7129,0.0724,0.3977,-2.7986,-0.8618,-0.2287,0.1292,2.6419,0.4305,0.1476
EM_Net,0.2091,1.893,0.0847,0.5546,-3.8371,-0.8244,-0.3174,0.1389,3.1858,0.4052,0.0999
ACWI_Net_PLN,0.1209,1.2714,0.0897,0.3604,-1.2476,-0.3336,-0.1647,0.0108,0.6982,0.7419,0.0964
MwigTR,0.2182,2.403,0.1182,0.4835,-3.8898,-0.6725,-0.2968,0.1952,3.7684,0.5417,0.1499
WIG20TR,0.22,2.2594,0.0731,0.5941,-3.2019,-0.3731,-0.3984,0.0078,1.1055,0.3321,0.1158


In [38]:
returns_summarystats(returns_usd,12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1491,1.4788,0.1,0.4254,-2.257,-0.7009,-0.2004,0.0014,1.3793,0.6706,0.1537
World_ex_USA,0.1682,1.7129,0.0543,0.4165,-2.7986,-0.6171,-0.2752,0.0571,1.6003,0.323,0.0992
EM_Net,0.2239,2.0825,0.0877,0.5565,-4.1,-0.8932,-0.3138,0.1422,3.1066,0.3914,0.1224


In [39]:
ols = lm(@formula(USA_Net ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

USA_Net ~ 1 + MwigTR

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00370211  0.00199595   1.85    0.0648  -0.000228753  0.00763297
MwigTR       0.470608    0.031373    15.00    <1e-36   0.408822     0.532395
──────────────────────────────────────────────────────────────────────────────
0.47171242027894034


In [47]:
ols = lm(@formula(MwigTR ~ USA_Net), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00149191  0.00293123   0.51    0.6112  -0.00428091  0.00726474
USA_Net      1.00235     0.0668212   15.00    <1e-36   0.870747    1.13395
─────────────────────────────────────────────────────────────────────────────
0.47171242027894


In [48]:
returns_lag = merge(returns_pln, TimeSeries.lead(returns_pln,1))
ols = lm(@formula(MwigTR ~  USA_Net_1 + USA_Net), returns_lag)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net_1 + USA_Net

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000679217  0.00298097   0.23    0.8199  -0.00519179  0.00655022
USA_Net_1    0.108191     0.0668057    1.62    0.1066  -0.0233824   0.239765
USA_Net      0.999189     0.0668003   14.96    <1e-35   0.867626    1.13075
──────────────────────────────────────────────────────────────────────────────
0.47753223872719375


In [54]:
ols = lm(@formula(World_ex_USA ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

World_ex_USA ~ 1 + MwigTR

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000668284  0.00218435   0.31    0.7599  -0.00363363  0.00497019
MwigTR       0.544382     0.0343344   15.86    <1e-39   0.476763    0.612001
──────────────────────────────────────────────────────────────────────────────
0.4993949288596782


In [55]:
ols = lm(@formula(EM_Net ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

EM_Net ~ 1 + MwigTR

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000866209  0.00289832   0.30    0.7653  -0.0048418  0.00657422
MwigTR       0.628798     0.0455567   13.80    <1e-31   0.539077   0.718518
─────────────────────────────────────────────────────────────────────────────
0.43052065797049277


In [56]:
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │
├────────────┼────────────┼──────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │
│ 2023-10-31 │ -0.0235616 │   -0.0431263 │   -0.039629 │
│ 2023-11-30 │  0.0895259 │    0.0898198 │   0.0769883 │
│ 2023-12-31 │  0.0456572 │    0.0532219 │   0.0383487 │
│ 2024-01-31

In [57]:
data = merge(returns_pln[:MwigTR], returns_usd[:USA_Net])
ols = lm(@formula(USA_Net ~ MwigTR), data)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

USA_Net ~ 1 + MwigTR

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00370211  0.00199595   1.85    0.0648  -0.000228753  0.00763297
MwigTR       0.470608    0.031373    15.00    <1e-36   0.408822     0.532395
──────────────────────────────────────────────────────────────────────────────
0.4717124202789381


In [58]:
# number of month wig20 > mwig40

n_wig20Gr = values(returns_pln[:WIG20TR] .> returns_pln[:MwigTR])  |> count
n_wig20Gr / length(returns_pln)


0.47244094488188976